# Импортируем необходимые библиотеки

In [ ]:
!pip install tmdbv3api

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from math import isnan, inf
from copy import deepcopy
from time import time_ns
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
# from tmdbv3api import TMDb
#from tmdbv3api import Movie
from tqdm.notebook import tqdm

import re
import torch
import requests

# Первый этап: создание профилей фильмов
- One-Hot Encoding для каждого жанра к которому принадлежит фильм, сумма таких векторов -- это закодированный жанр фильма
- Дата выхода фильма на экране

## Загрузка необходимых таблиц

In [ ]:
movies = pd.read_csv("/kaggle/input/movielensfull/ml-latest/movies.csv")
links = pd.read_csv("/kaggle/input/movielensfull/ml-latest/links.csv")

## Переводим жанры для фильма в формат One-Hot Encoding

In [ ]:
genres = set()

for row in movies.itertuples():
    genres.update(getattr(row, "genres").split("|"))

genres = list(genres)
genres.remove('(no genres listed)')

def genresToVec(genre, text: str) -> np.array:
    if genre in text:
        return 1

    return 0

for genre in genres:
    movies[genre] = movies["genres"].apply(lambda x: genresToVec(genre, x))

## Вытаскиваем год выхода из названия фильма

In [ ]:
def extract_year(text: str) -> int:
    search = re.search("\(\d\d\d\d\)", text)
    
    if search is None:
        return -1
    else:
        return search.group(0).replace("(", "").replace(")", "")

movies["year"] = movies["title"].apply(extract_year)

## Проблема: год есть в названии не всех фильмов

- IMDB api воспользоваться не получится, так как аккаунт в AWS из России не завести
- TMDB api было бы хорошим вариантом, но в табличке не для всех фильмов есть tmdbId, а те которые есть не все верные
- __в качестве решния этой проблемы воспользуемся комбинированным подходом: используем поиск по id где это возможно, а иначе воспользуемся поиском по названию с помощью пакета tmdbv3api и, наконец, остатки разметим вручную__

In [ ]:
tmdb = TMDb()
tmdb.api_key = '2fb17a68b167a63f1ce5f42308454db6'
movie = Movie()

### Заполняем пропуски по id

In [ ]:
tmdb = TMDb()
tmdb.api_key = '2fb17a68b167a63f1ce5f42308454db6'
movie = Movie()


def get_year_by_id(tmdbId) -> str:
    
    if isnan(tmdbId):
        return -1
    try:
        res = movie.details(int(tmdbId))["release_date"][:4]
        if len(res) < 4:
            return -1
        else:
            return res
    except Exception as e:
        return -1    


moviesWL = pd.merge(movies, links, on="movieId", how="left")
movies["year"][movies["year"] == -1] = moviesWL[moviesWL["year"] == -1]["tmdbId"].apply(get_year_by_id)

### Заполняем пропуски по названию

In [ ]:
def get_year_by_title(title: str) -> str:
    try:
        search = movie.search(title)[0]["release_date"][:4]
        
        if len(search) < 4:
            return -1
        else:
            return search
    except Exception as e:
        return -1

movies["year"][movies["year"] == -1] = movies[movies["year"] == -1]["title"].apply(get_year_by_title)

### Заполняем остатки пропусков вручную

In [ ]:
for row in movies[movies["year"] == -1].itertuples():
    print(getattr(row, "title"))
    movies.loc[getattr(row, "Index"), "year"] = input()
    

### Проверяем датасет на отсутствие ошибок

In [ ]:
indexes = []

for row in movies.itertuples():
    if len(str(getattr(row, "year"))) < 4:
        indexes.append(getattr(row, "Index"))

assert len(movies.iloc[indexes]) == 0, "Что-то не так, перепроверить данные"

### Сохраняем датасет в удобном для дальнейшего использования формате

In [ ]:
movies["year"] = movies["year"].apply(int)
movies.to_csv("movie_profiles.csv", index=False)

### Сохраняем файл для дальнейшего использования

In [ ]:
movies.to_csv("movie_profiles.csv", index=False)

# Второй этап: составляем профили пользователей

- 19-ти мерный вектор средних оценок по жанрам
- "любимый год": год с самой выской средней оценкой

## Загружаем необходимые датасеты

In [ ]:
movies = pd.read_csv("/kaggle/input/movie-profiles-final/movie_profiles.csv")
ratings = pd.read_csv("/kaggle/input/movielensfull/ml-latest/ratings.csv")

## Ассоциируем отзыв с жанрами фильма на который он оставлен

In [ ]:
reviews_with_genres = pd.merge(ratings.drop(columns=["timestamp"]), movies.drop(columns=["title", "genres"]), on="movieId", how="left")

for genre in genres:
    reviews_with_genres[genre] = reviews_with_genres[genre] * reviews_with_genres["rating"]

## Создаём "профиль" юзера: вектор средних по жанрам

In [ ]:
def create_user_profile(user):
    
    if user.iloc[0]["userId"] % 1000 == 0.0:
        print(user.iloc[0]["userId"], end="\r")
        
    d = {}
    
    for genre in genres:
        zeros = np.count_nonzero(user[genre] == 0)
        
        if zeros == user[genre].shape[0]:
            d[genre] = 0
        else:
            d[genre] = np.sum(user[genre]) / (user[genre].shape[0] - zeros)
    
    return pd.Series(d, index=genres)
        

user_profiles = reviews_with_genres.groupby("userId").apply(create_user_profile)
user_profiles.reset_index(inplace=True)

## Выявляем любимый год юзера: год с самой высокой средней оценкой

In [ ]:
res = reviews_with_genres.groupby(["userId", "year"], as_index=False).agg({"rating": "mean"})
user_fav_years = res.iloc[res.groupby("userId").agg({"rating": "idxmax"})["rating"]]

## Добавляем любимый год в профиль и сохраняем датасет

In [ ]:
user_profiles = pd.merge(user_profiles, user_fav_years[["userId", "year"]], on="userId", how="left")
user_profiles.to_csv("user_profiles.csv", index=False)

# Третий этап: архитектура и обучение нейросети

Архитектура сети представлена в ячейке ниже

## EmbeddingNet

In [ ]:
class EmbeddingNet(nn.Module):
    def __init__(self, user_length=20, movie_length=20, output_length=50):
        super().__init__()
        
        self.user_encoder = nn.Sequential(
            nn.Linear(user_length, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, output_length)
        )
        
        self.movie_encoder = nn.Sequential(
            nn.Linear(movie_length, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.Linear(128, output_length)
        )
        
        self.cosine = nn.CosineSimilarity(dim=2)
    
    def forward(self, user, movie):
        
        user_embedding = self.user_encoder(user)
        movie_embedding = self.movie_encoder(movie)

        similarity = self.cosine(user_embedding, movie_embedding)
        return (similarity + 1) * 2.5
        

## Создаём pytorch Dataset

In [ ]:
class RatingsDataset(Dataset):
    
    def __init__(self, ratings, user_profiles, movie_profiles):
        self.ratings = ratings
        self.user_profiles = user_profiles
        self.movie_profiles = movie_profiles
        self.columns = ['Action', 'Mystery', 'Documentary', 'War', 'Comedy',
       'Musical', 'Film-Noir', 'Adventure', 'Drama', 'Fantasy', 'Romance',
       'Animation', 'Crime', 'Children', 'Thriller', 'IMAX', 'Sci-Fi',
       'Western', 'Horror', 'year']
    
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        review = self.ratings.iloc[idx]
        user = torch.tensor(
            self.user_profiles.loc[self.user_profiles["userId"] == review["userId"], self.columns].to_numpy(),
            dtype=torch.float32
        )
        movie = torch.tensor(
            self.movie_profiles.loc[self.movie_profiles["movieId"] == review["movieId"], self.columns].to_numpy(),
            dtype=torch.float32
        )
        return user, movie, torch.tensor(review["rating"], dtype=torch.float32)

## Загружаем необходимые датасеты

In [ ]:
ratings = pd.read_csv("/kaggle/input/movielensfull/ml-latest/ratings.csv")
ratings = ratings.groupby("rating", group_keys=False).apply(lambda x: x.sample(frac=0.0075, random_state=42))
user_profiles = pd.read_csv("/kaggle/input/user-profiles-final/user_profiles.csv")
movie_profiles = pd.read_csv("/kaggle/input/movie-profiles-final/movie_profiles.csv")

## Проводим train/val/test split и скалируем данные

In [ ]:
ratings_train, ratings_val = train_test_split(ratings, test_size=0.4, random_state=42, stratify=ratings["rating"])
ratings_val, ratings_test = train_test_split(ratings_val, test_size=0.5, random_state=42, stratify=ratings_val["rating"])

In [ ]:
genres = ['Action', 'Mystery', 'Documentary', 'War', 'Comedy',
       'Musical', 'Film-Noir', 'Adventure', 'Drama', 'Fantasy', 'Romance',
       'Animation', 'Crime', 'Children', 'Thriller', 'IMAX', 'Sci-Fi',
       'Western', 'Horror']

for genre in genres:
    scaler = MinMaxScaler()
    user_profiles[[genre]] = scaler.fit_transform(user_profiles[[genre]])
    
user_scaler = StandardScaler()
user_profiles[["year"]] = user_scaler.fit_transform(user_profiles[["year"]])

movie_scaler = StandardScaler()
movie_profiles[["year"]] = movie_scaler.fit_transform(movie_profiles[["year"]])

## Подготавливаем pytorch даталоадеры

In [ ]:
train_ds = RatingsDataset(ratings_train, user_profiles, movie_profiles)
val_ds = RatingsDataset(ratings_val, user_profiles, movie_profiles)
test_ds = RatingsDataset(ratings_test, user_profiles, movie_profiles)

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=2028, shuffle=True, num_workers=4, pin_memory=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=2048, shuffle=True, num_workers=4, pin_memory=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=2048, shuffle=True, num_workers=4, pin_memory=True)

## Подготавливаем модель к обучению

### По возможности будем обучать на GPU

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

### Инициализируем модель, оптимизатор. В качестве loss-функции была выбрана MSE, она же выступает в качестве метрики качества

In [ ]:
model = EmbeddingNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
loaders = { "train": train_dl, "val": val_dl }
history = { "train": [], "val": [] }

best_model = deepcopy(model.state_dict())
best_loss = inf

In [ ]:
max_epochs = 20

for epoch in tqdm(range(max_epochs)):
    
    start = time_ns() * 1e-9
    
    train_losses = []
    validation_losses = []
    
    if epoch == 10:
        optimizer.param_groups[0]["lr"] = 0.0001
        
    if epoch == 15:
        optimizer.param_groups[0]["lr"] = 0.00001
    
    for k, dataloader in loaders.items():

        for user, movie, rating in tqdm(dataloader, total=len(dataloader)):
            
            user = user.to(device)
            movie = movie.to(device)
            rating = rating.to(device)
            
            optimizer.zero_grad()
            
            if k == "train":
                model.train()
                output = model(user, movie)
            else:
                model.eval()
                with torch.no_grad():
                    output = model(user, movie)
            
            if k == "train":
                loss = criterion(torch.flatten(output), rating)
                loss.backward()
                optimizer.step()
                train_losses.append(loss.detach().to("cpu").item())
            else:
                with torch.no_grad():
                    loss = criterion(torch.flatten(output), rating)
                    validation_losses.append(loss.detach().to("cpu").item())
        
    train_avg_loss = sum(train_losses) / len(train_losses)
    history["train"].append(train_avg_loss)
    
    val_avg_loss = sum(validation_losses) / len(validation_losses)
    history["val"].append(val_avg_loss)
    
    if val_avg_loss < best_loss:
        best_loss = val_avg_loss
        best_model = deepcopy(model.state_dict())
    
    end = time_ns() * 1e-9    
    print(f"Iteration №{epoch + 14}. Train MSE: {train_avg_loss}. Val MSE: {val_avg_loss}. Time used: {end - start}.")
        

In [ ]:
X = [i for i in range(1, len(history["train"]) + 1)]

plt.plot(X, history["train"])
plt.plot(X, history["val"])
plt.show()

In [ ]:
torch.save(best_model, "model_val_200k_strata.pt")

In [ ]:
model.load_state_dict(best_model)
model.eval()

In [ ]:
def score(model, dataloader, device, criterion):
    
    model.to(device)
    model.eval()
    
    losses = []
    
    for user, movie, rating in tqdm(dataloader, total=len(dataloader)):
            
        user = user.to(device)
        movie = movie.to(device)
        rating = rating.to(device)
            
                
        with torch.no_grad():
            output = model(user, movie)
            loss = criterion(torch.flatten(output), rating)
            losses.append(loss.detach().to("cpu").item())
    
    return sum(losses) / len(losses)

In [ ]:
score(model, test_dl, device, criterion)

На тестовых данных модель получила score: 0.7643040248325893

# Четвёртый этап: получения эмбеддингов

## Создаём и сохраняем эмбеддинги

In [ ]:
embeddings = model.movie_encoder(torch.tensor(movie_profiles[genres + ["year"]].to_numpy(), dtype=torch.float32).to(device)).detach().to("cpu").numpy()
np.save("embeddings.npy", embeddings)

## Обучаем объект NearestNeighbours на полученных эмбеддингах

In [ ]:
knn = NearestNeighbors(n_neighbors=10, metric="cosine").fit(embeddings)

## Пример использования

- создаём с помощью user_encoder-а из модели эмбеддинг юзера
- с помощью метода NearestNeighbor kneighbors получаем индексы N ближайших фильмов
- по этим индексам получаем фильмы с помощью таблицы movies

In [ ]:
user_embedding = model.user_encoder(torch.tensor(user_profiles.iloc[0][genres + ["year"]].to_numpy(), dtype=torch.float32).to(device)).detach().to("cpu").numpy()
_, indices = knn.kneighbors([user_embedding])

In [ ]:
movies = pd.read_csv("/kaggle/input/movielensfull/ml-latest/movies.csv")
movies.iloc[indices[0]]

# Пятый этап: пути дальнейшего развития

- Во-первых, на мой взгляд, у нас мало данных для обучения. К сожалению, у меня нет доступа к IMDB API, а TMDB API имеет лимит запросов кратно меньший количества фильмов в датасете, но "given enough time" можно было бы достать следующие данные: 

    1. Популярность фильма (по сути, рейтинг на сайте, т.е. число) 

    2. Страна выпуска/язык оригинала (можно представить в виде вектора таким образом, чтобы языки схожих культур находились бы ближе друг к другу) 

    3. Бюджет (число) 

    4. Длина фильма (число) 

    5. Выручка фильма (число) 

    6. Список актёров, режиссёр и сценарист (можно представить в виде векторов по аналогии с word2vec исходя из того, как часто они встречаются вместе/в фильмах одного жанра etc. 
  

- Во-вторых, мои эксперименты показали, что обучение большей модели на большей выборке данных (с сохранением распределения классов) даёт лучшие результаты. В идеале можно было бы обучить модель на большей выборке, в том числе и на всём датасете, если бы на это было время и/или мощности. 

- В-третьих, можно было бы поэкспериментировать с различными функциями активации.

- В четвёртых, можно было бы дообучать модель на нескольких выборках, ещё уменьшить learning rate